In [1]:
# install all the required packages for the demo
# !CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install llama-cpp-python
# !pip install pypdf sentence-transformers chromadb langchain

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

Next, initialize the langchain `CallBackManager`. This handles callbacks from Langchain and for this example we will use token-wise streaming so the answer gets generated token by token when Llama is answering your question.

In [4]:
# for token-wise streaming so you'll see the answer gets generated token by token when Llama is answering your question
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


Set up the Llama 2 model. 

Replace `<path-to-llama-gguf-file>` with the path either to your downloaded quantized model file [here](https://drive.google.com/file/d/1afPv3HOy73BE2MoYCgYJvBDeQNa9rZbj/view?usp=sharing), 

or to the `ggml-model-q4_0.gguf` file built with the following commands:

```bash
git clone https://github.com/ggerganov/llama.cpp
cd llama.cpp
python3 -m pip install -r requirements.txt
python convert.py <path_to_your_downloaded_llama-2-13b_model>
./quantize <path_to_your_downloaded_llama-2-13b_model>/ggml-model-f16.gguf <path_to_your_downloaded_llama-2-13b_model>/ggml-model-q4_0.gguf q4_0

```
For more info see https://python.langchain.com/docs/integrations/llms/llamacpp

In [5]:

llm = LlamaCpp(
    model_path="/mnt/DATASSD2/LargeModels/LLaMa-2/llama-2-13b-gguf/ggml-model-q4_0.gguf",
    n_gpu_layers=-1,
    temperature=1,
    top_p=0.9,
    n_ctx=7000,
    callback_manager=callback_manager, 
    verbose=True,
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3090, compute capability 8.6
llama_model_loader: loaded meta data with 16 key-value pairs and 363 tensors from /mnt/DATASSD2/LargeModels/LLaMa-2/llama-2-13b-gguf/ggml-model-q4_0.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:               output_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:                    output.weight q6_K     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_q.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    5:              blk.0.attn_v.weight q4_0     [  5120,  5120,     1,

In [6]:

from langchain.vectorstores import Chroma

# embeddings are numerical representations of the question and answer text
from langchain.embeddings import HuggingFaceEmbeddings

# use a common text splitter to split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

# EJEMPLO 1

In [7]:
a = '''
responsabilidad social corporativa de inditex
Colaboramos para generar un impacto positivo
Nos encontramos ante un cambio de paradigma en el que la colaboración de diversos actores puede lograr una transformación sostenible de nuestra industria. Por ello, todos los que formamos Inditex colaboramos con universidades, organizaciones sociales, sindicatos y otras empresas para avanzar hacia un modelo circular, en el que nuestros proveedores crezcan y podamos seguir generando valor en las comunidades en las que desarrollamos nuestra actividad, con el objetivo de lograr un impacto positivo para las personas y el planeta.
Colaboramos con nuestras personas, motor de esa transformación
Nuestros equipos son el verdadero motor de nuestra transformación. Por eso, impulsamos oportunidades de desarrollo y formación que contribuyan a su desarrollo personal y profesional.
Colaboramos con nuestros clientes, los grandes impulsores
La relación con nuestros clientes es la base de nuestro modelo de negocio. Sus percepciones y comentarios son claves a la hora de impulsarnos para conseguir una industria más sostenible.
Colaboramos con nuestros accionistas, aliados para transformar
Potenciamos una comunicación transparente y abierta con nuestros accionistas, favoreciendo su participación en la toma de decisiones, clave para seguir avanzando en la transformación del sector.
Colaboramos para transformar a través de una gestión sostenible de nuestros productos
Ofrecer a nuestros clientes productos sostenibles y avanzar hacia un modelo circular es vital para conseguir una transformación de nuestra industria.
Colaboramos para que nuestros proveedores crezcan
Una relación fluida con nuestros proveedores es clave para que avancen con nosotros hacia una cadena de suministro más sostenible, dando respuestas a los retos y desafíos de los diferentes mercados en los que operamos.
Colaboramos para preservar el planeta
Nuestro compromiso con la sostenibilidad impregna todas las fases de la cadena de valor. Reducir el impacto ambiental y contribuir a una industria más sostenible son nuestros principales objetivos.
Colaboramos para generar valor en las comunidades
Nuestra misión trasciende al plano económico y pasa por aportar valor y contribuir al desarrollo social. Por ello, en 2021 colaboramos con 725 iniciativas sociales que beneficiaron a 2,2 millones de personas.
Responsabilidad y transparencia fiscal
El estricto cumplimiento de nuestras obligaciones fiscales responde también a nuestro compromiso con la creación de valor y al desarrollo de un impacto social positivo allí donde estamos presentes.
Buen gobierno, cultura ética corporativa y sólida arquitectura de Compliance
Transmitir la cultura ética corporativa a todos los grupos de interés resulta esencial para Inditex, que cuenta con un sistema normativo interno que favorece el desarrollo de un modelo ético, eficiente y competitivo.
Gestión responsable del riesgo
Una gestión eficaz del riesgo en todos los niveles del Grupo nos permite mantener un buen desempeño y genera un rendimiento empresarial sostenible a lo largo del tiempo, necesario para transformar.
'''

In [8]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("inditex.pdf")
documents = loader.load()

In [9]:
# split the loaded documents into chunks 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

# create the vector db to store all the split chunks as embeddings
embeddings = HuggingFaceEmbeddings()
vectordb = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings,
)

/home/pjvazquez/miniconda3/envs/llama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# use another LangChain's chain, RetrievalQA, to associate Llama with the loaded documents stored in the vector db
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [11]:

question = "diseña unas acciones de responsabilidad social corporativa para inditex"
result = qa_chain({"query": question})


Based on the context provided, here are some possible actions for Inditex's corporate social responsibility (CSR) initiatives:

1. Collaboration with local communities: Continuing the collaboration with universities, organizations, and other stakeholders to advance towards a circular model, benefiting the communities where they operate. This could include initiatives such as training programs for sustainable fashion, waste reduction, and community engagement.
2. Sustainable supply chain: Strengthening relationships with suppliers to promote their growth and development while maintaining ethical and responsible practices throughout the supply chain. This could involve supporting sustainable materials sourcing, reducing waste and emissions, and ensuring fair labor practices.
3. Environmental preservation: Implementing measures to reduce the environmental impact of their operations, such as reducing energy consumption, implementing recycling programs, and promoting sustainable packaging.


llama_print_timings:        load time =      44.92 ms
llama_print_timings:      sample time =      35.47 ms /   256 runs   (    0.14 ms per token,  7216.96 tokens per second)
llama_print_timings: prompt eval time =    3292.24 ms /   975 tokens (    3.38 ms per token,   296.15 tokens per second)
llama_print_timings:        eval time =    3535.36 ms /   255 runs   (   13.86 ms per token,    72.13 tokens per second)
llama_print_timings:       total time =    7305.77 ms


# EJEMPLO 2
- extrater texto de un html

In [12]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [13]:
url = "https://static.inditex.com/annual_report_2021/es/impacto-positivo.html"
url = "https://www.linkedin.com/company/inditex/?originalSubdomain=es"
url = "https://www.repsol.com/es/sostenibilidad/index.cshtml"
html = urlopen(url)

In [14]:
soup = BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [15]:
text = soup.text
text_2 = text.strip()
text_2 = text_2.replace("\n", " ")
text_2 = text_2.replace("\t", " ")
text_2 = text_2.replace("\r", " ")
text_2 = text_2.replace("  ", " ")

In [16]:
print(len(text_2))

15818


In [17]:
question = "extract and sumarize text from:" + text_2
result = qa_chain({"query": question})

Llama.generate: prefix-match hit


ost/1
1ot/impododeostoseostoresome 1ore1ide1ouslyignouchose laodootsodronne, 1ode burorose 1ose burdoouslyingostodoresody 1oresode 1ores 1ey/1                ning 1 1oresodo 1ore 1ore 1oreably'ayably lautor/standsiding'ontonally, "bour 1oresosously/red 1to the 1 1
Osli laorto... 21103ody2toostowomooostotsooresurtheat
1ore 1-1.
1ore 1ore 1ore
1
Stores
1
br The the la 1wards The act spotsaking
takesoresode laisachreadtoaytosiding thisdo firstactally 2 slip s' 1omono 1omoo (tosing 4ones laontazode. " O. 12the de 1 on adi' shore 1. 12 ci12152 1


llama_print_timings:        load time =      44.92 ms
llama_print_timings:      sample time =      30.84 ms /   256 runs   (    0.12 ms per token,  8300.91 tokens per second)
llama_print_timings: prompt eval time =   23240.61 ms /  5462 tokens (    4.25 ms per token,   235.02 tokens per second)
llama_print_timings:        eval time =    5217.98 ms /   255 runs   (   20.46 ms per token,    48.87 tokens per second)
llama_print_timings:       total time =   29337.22 ms
